In [1]:
import pandas as pd
import numpy as np

# 假設資料來源
df = pd.read_csv("all_dataset.csv")

# Step 1. 移除 type 為 "接不到" 的列
df = df[df['type'] != '接不到']

# 建立一個布林遮罩 (mask)
# df.duplicated(subset=['rally_id'], keep='last')
# 會標記出所有在 'rally_id' 群組中 "不是最後一筆" 的資料 (標記為 True)
mask_not_last_in_rally = df.duplicated(subset=['rally_id'], keep='last')

# 使用 .loc 索引，將這些 "不是最後一筆" 資料的 'getpoint_player' 欄位設為 NaN
df.loc[mask_not_last_in_rally, 'getpoint_player'] = np.nan
print("✅ 'getpoint_player' 處理完畢。")

# Step 2. 若 lose_reason 或 win_reason 包含「對手」則改為 NaN
for col in ['lose_reason', 'win_reason']:
    df[col] = df[col].apply(lambda x: np.nan if isinstance(x, str) and "對手" in x else x)

# Step 3. 每場比賽的每個 set 最後插入一筆新資料
# ----------------------------------------------------

# 建立新的 DataFrame 來放處理結果
new_df = pd.DataFrame(columns=df.columns)

# 分組：每場比賽 + 該比賽的每個 set
for (match_id, set_id), group in df.groupby(['match_id', 'set'], sort=False):
    # 先把該組原本資料加入新 DataFrame
    new_df = pd.concat([new_df, group], ignore_index=True)

    # 取最後一筆並複製
    last_row = group.iloc[-1].copy()
    last_row['score_status'] += 1

    # 修改分數
    if last_row['player_score'] == 20:
        last_row['player_score'] += 1
    else:
        last_row['opponent_score'] += 1

    # 立即插入到該組最後
    new_df = pd.concat([new_df, pd.DataFrame([last_row])], ignore_index=True)

# 輸出結果
new_df.to_csv("processed_data.csv", index=False)
print("✅ 已完成：每場比賽的每個 set 都在最後插入一筆新資料。")


✅ 'getpoint_player' 處理完畢。


C:\Users\Danie\AppData\Local\Temp\ipykernel_6340\714223583.py:32: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  new_df = pd.concat([new_df, group], ignore_index=True)


✅ 已完成：每場比賽的每個 set 都在最後插入一筆新資料。


In [2]:
import pandas as pd
import sqlite3

# 讀取 CSV 檔
csv_file = "processed_data.csv"
df = pd.read_csv(csv_file)

# 建立 SQLite 資料庫（若不存在會自動建立）
db_file = "processed_data.db"
conn = sqlite3.connect(db_file)

# 將 DataFrame 寫入 SQLite，表格名稱可自訂
table_name = "match_data"
df.to_sql(table_name, conn, if_exists="replace", index=False)

# 確認寫入完成
print(f"✅ 已將 {csv_file} 匯入 SQLite 資料庫 {db_file}，表格名稱：{table_name}")

# （可選）查看前 5 筆資料確認
print(pd.read_sql(f"SELECT * FROM {table_name} LIMIT 5;", conn))

# 關閉連線
conn.close()


✅ 已將 processed_data.csv 匯入 SQLite 資料庫 processed_data.db，表格名稱：match_data
   rally  ball_round      time  frame_num  score_status  player_score  \
0      1           1   0:07:39    11496.0           0.0           0.0   
1      1           2  00:07:43    11582.0           0.0           0.0   
2      2           1   0:07:55    11881.0           1.0           1.0   
3      2           2   0:07:56    11900.0           1.0           1.0   
4      2           3   0:07:56    11921.0           1.0           1.0   

   opponent_score          player  server type  ...  player_move_area  \
0             0.0    Kento MOMOTA     1.0  發短球  ...              11.0   
1             0.0  CHOU Tien Chen     3.0   長球  ...              18.0   
2             0.0    Kento MOMOTA     1.0  發短球  ...              11.0   
3             0.0  CHOU Tien Chen     2.0  推撲球  ...              10.0   
4             0.0    Kento MOMOTA     2.0   殺球  ...               7.0   

   moving_x  moving_y  landing_court_number  ball_